## 바른 형태소 분석기 활용 4(사용자 사전 사용하기)
(이미 설치를 완료하신 분들은 아래 '샘플 텍스트 파일 다운로드' 부분부터 실행하면 됩니다.)

바른 설치
바른을 설치합니다.

바른의 리눅스 설치본을 다운로드 받습니다.

In [ ]:
!curl -LJks -H "uname:$(uname -a)" https://bareun.ai/api/get -o bareun-linux.deb
!ls

bareun-linux.deb  drive  sample_data


In [ ]:
!uname -a

Linux 3ebcedc2984c 5.15.107+ #1 SMP Sat Apr 29 09:15:28 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux


**바른**을 설치합니다.

In [ ]:
!dpkg -i bareun-linux.deb

(Reading database ... 122541 files and directories currently installed.)
Preparing to unpack bareun-linux.deb ...
Unpacking bareun (2.1.0) over (2.0.4) ...


아래는 바른 모델이 GPU를 사용하기 위해서 다운받는 tensorflow 라이브러리입니다.

상단의 메뉴탭의 "런타임 > 런타임 유형 변경 > 하드웨어 가속기 > GPU 선택 > 저장"을 설정하시면 모델의 추론 속도가 더 빨라집니다. 단, 무료버전의 경우 coloab의 일일 사용 제한을 초과하지 않아야 합니다.

기본적으로 CPU를 사용해도 작동합니다.

In [ ]:
!curl -O https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  392M  100  392M    0     0  21.4M      0  0:00:18  0:00:18 --:--:-- 23.9M


In [ ]:
!tar -C /opt/bareun -xzf libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

## 환경 설정하기
아래 코드는 BAREUN_ROOT와 LD_LIBRARY_PATH를 설정하고, /opt/bareun/bin/bareun을 백그라운드로 실행하는 코드입니다.

In [ ]:
%env BAREUN_ROOT="/opt/bareun"
%env LD_LIBRARY_PATH="/opt/bareun/lib"
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" nohup /opt/bareun/bin/bareun&


env: BAREUN_ROOT="/opt/bareun"
env: LD_LIBRARY_PATH="/opt/bareun/lib"
nohup: appending output to 'nohup.out'


In [ ]:
!ps -ef | grep bareun

root        8111       1  1 06:32 ?        00:00:43 /opt/bareun/bin/bareun
root       24572     552  0 07:37 ?        00:00:00 /bin/bash -c ps -ef | grep bareun
root       24574   24572  0 07:37 ?        00:00:00 grep bareun


API키 발급 바른 홈페이지(https://bareun.ai/) 에 회원가입해서 API키를 받습니다.

API키 복사
발급 받은 API키를 -reg 인자 다음에 입력하여 등록합니다.

In [ ]:
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" /opt/bareun/bin/bareun -reg "YOUR-API-KEY"

Use config file: /opt/bareun/config/bareun.json

------
200 OK
This apikey is already registered.


## 바른 사용하기
**PIP 패키지 설치**


In [ ]:
!pip install -U bareunpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
from bareunpy import Tagger
API_KEY = "YOUR-API-KEY"
tagger = Tagger(API_KEY, "localhost", 5656)

## 샘플 텍스트 파일 다운로드

* legal_sentences.txt: 약 3천 문장으로 구성된 샘플 법률(민사) 텍스트 파일입니다(출처:https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=580).
* legal_words.txt: 법률 용어 사전의 일부입니다.

In [ ]:
!pip install -U gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
!gdown  https://drive.google.com/file/d/1LOLOzX8xP_Mtsy3eHN6tmNHuHbWoPgyK/view?usp=sharing -O legal_sentences.txt --fuzzy
!gdown  https://drive.google.com/file/d/1qPPtcsq1_z8zwT3g4TzEWs5nAgWtTd9-/view?usp=sharing -O legal_words.txt --fuzzy

Downloading...
From: https://drive.google.com/uc?id=1LOLOzX8xP_Mtsy3eHN6tmNHuHbWoPgyK
To: /content/legal_sentences.txt
100% 1.04M/1.04M [00:00<00:00, 108MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qPPtcsq1_z8zwT3g4TzEWs5nAgWtTd9-
To: /content/legal_words.txt
100% 827/827 [00:00<00:00, 3.80MB/s]


## 사용자 사전 사용 예제

* 사용자 사전은 형태소 분석기가 원래 알고 있지 못하는 단어나 표현을 인식하도록 도와줍니다. 사용자 사전의 일반적인 사용 목적은 다음과 같습니다.
  * 특수한 도메인의 단어 인식
     * 의료, 법률, IT, 문학, 과학 등 특정 도메인에서만 사용되는 어려운 단어를 등록하여 인식률을 높입니다.
  * 이름, 기업명, 제품명 등 고유명사 인식
  * 특정 단어에 대한 분석(분절) 방법 변경
    * 예를 들어, GS25는 원래 "GS/SL+25/SN" 이렇게 분석되지만 사용자 사전에 등록하면 "GS25/NNG" 이렇게 분석되어 특정한 명사를 찾을 때 유용합니다.

* 이 예제에서는 법률 도메인의 문장을 분석하여 사용자 사전을 등록했을 때 어떻게 분석되는지 확인해봅니다.




In [52]:
# 문장 로딩
with open('legal_sentences.txt', 'r') as file:
  legal_sentences = [line.strip() for line in file]
  print(f'legal_sentenses[:10]: {legal_sentences[:10]}')
# 단어 로딩
with open('legal_words.txt', 'r') as file:
  legal_words = [line.strip() for line in file]
  print(f'legal_words[:10]: {legal_words[:10]}')

legal_sentenses[:10]: ['피고 측 차량이 신호대기 중에 핸들이 우측으로 틀어져 있었고 이후 그대로 진행하다가 우측에 있던 원고 측 차량과 충돌하면서 사고가 발생하기는 하였으나, 피고 측 측 차량과 그 뒤의 차량이 신호대기로 정차하고 있는 중에 원고 측 차량이 한 개 차로의 우측에 있는 좁은 빈 공간을 통하여 무리하게 진행한 행위가 사고 발생에 영향을 끼쳤다.', '피고 측 차량이 좌측 방향지시등을 켜놓았다고 하여 원고 측 차량이 한 개 차로의 우측에 있는 좁은 빈 공간을 통하여 진행하는 행위가 정당화된다고 볼 수 없다.', '피고 측 차량의 바퀴가 상당한 정도로 우측으로 꺾여 있었으므로 원고 측 차량은 이를 주의 깊게 보았다면 우측의 좁은 빈공간을 진행하는 것을 더욱 주의해야 하였을 것으로 보인다.', '원고 측 과실비율 40%, 피고 측 과실비율 60%로 봄이 타당하다.', '구상금액은 1,278,700원(= 총 손해액 2,964,500원 X 피고 측 과실비율 60% - 자기부담금 500,000원)이다.', '원고는 피고 또는 000 중 어느 누구에게나 보증채무의 이행으로 출연한 금액 전부에 대해 구상권을 행사할 수 있다.', '가사 원고가 피고 주장과 같이 연대채무자인 000의 회생절차에서 회생채권 신고를 제때 하지 않았다고 하더라도, 피고에 대한 관계에서 이를 두고 위법한 행위라거나, 그로써 피고가 어떠한 손해를 입었다고 볼 수 없다.', '따라서 피고의 상계 항변은 그 자동채권의 존재를 인정할 수 없으므로, 더 나아가 살펴 볼 것도 없이 이유 없다.', '그러나 피고가 주장하는 사정만으로는 원고의 청구가 신의칙에 반하는 권리남용이라고 보기 어렵고, 달리 이를 인정할 증거가 없으므로, 피고의 위 주장도 받아들일 수 없다.', '2006. 8. 22. 피고를 시공사로 선정한 주민총회 의결이 있었더라도 이는 무효이고, 그에 기하여 체결된 이 사건 도급약정 역시 무효이다.']
legal_words[:10]: ['기소유예', '현장검증', '현

In [51]:
import json

# 사용자 사전 저장
cust_dic = tagger.custom_dict("my_dict_01") # 사전이름 영문, 숫자, 기호('-', '_')만 사용 가능

# 복합명사 사전 단어 저장: 위에서 불러온 legal_words 리스트를 set type으로 저장합니다.
cust_dic.copy_cp_set(set(legal_words))
cust_dic.update()

# 이전 사용자 사전 불러오기
cust_dict2 = tagger.custom_dict("my_dict_01")
cust_dict2.load()

tagger.set_domain('my_dict_01')

# 출력 결과를 텍스트 파일로 저장합니다.
with open('output_lagal_words.txt', 'w', encoding='utf-8') as f:
  # 문장 분석
  idx = 1
  for sent in legal_sentences:
    # 형태소 분석 결과를 딕셔너리 형태로 저장
    res = tagger.tags([sent]) 
    res_dict = json.loads(res.as_json_str())

    # 사용자 사전에 등록된 단어인 경우, 형태소와 문장을 출력합니다.
    for token in res_dict['sentences'][0]['tokens']:
        custom_token = [f"{t['text']['content']}/{t['tag']}"+", "+f"{t['outOfVocab']}" \
                        for t in token['morphemes'] if t['outOfVocab'] == 'IN_CUSTOM_DICT']
        if len(custom_token) != 0:
            print(f"{idx}: {custom_token}: {sent}")
            f.write(f"{idx}: {custom_token}: {sent}\n")
            idx += 1


1: ['채무불이행/NNG, IN_CUSTOM_DICT']: 이 사건 제 4~12 게시 글의 내용 중 일부 감정적이고 거친 표현이 포함되어 있다고 하더라도 그 게시행위 역시 원고의 사회적 평가를 저해할 정도에 이르는 비방 또는 명예훼손 행위에 해당하지 않거나 표현의 자유의 한계를 벗어나지 아니한 행위로서 위법성이 조각되어 채무불이행이 성립하지 않는다고 봄이 타당하다.
2: ['채무불이행/NNG, IN_CUSTOM_DICT']: 이 사건 비방금지 약정 위반에 따른 채무불이행이 성립하지 않는다고 봄이 타당하다.
3: ['채무불이행/NNG, IN_CUSTOM_DICT']: 뿐만 아니라, 설령 원고의 비밀에 해당한다고 하더라도 위 나) 항에서 본 사정들에 비추어 그 제보행위는 공공의 이익을 위한 것으로서 그 표현내용이나 방법 등이 부당하다고 볼 수 없어 위법성이 조각된다고 할 것이므로, 이 사건 비밀누설금지 약정 위반에 따른 채무불이행도 성립하지 않는다고 봄이 타당하다.
4: ['채무불이행/NNG, IN_CUSTOM_DICT']: 이 사건 비방금지 약정 위반에 따른 채무불이행이 성립하지 않는다고 봄이 타당하다.
5: ['채무불이행/NNG, IN_CUSTOM_DICT']: 설령 원고의 비밀에 해당한다고 하더라도 위 나)항에서 본 사정들에 비추어 그 제보행위는 공공의 이익을 위한 것으로서 그 표현내용이나 방법 등이 부당하다고 볼 수 없어 위법이 조각된다고 할 것이므로, 이 사건 비밀누설금지 약정 위반에 따른 채무불이행이 성립하지 않는다고 봄이 타당하다.
6: ['채무불이행/NNG, IN_CUSTOM_DICT']: 설령 피고의 이 부분 제보 내용이 원고의 명예나 신용을 훼손할 수 있는 내용이라 하더라도, 이는 공공의 이익을 위한 것으로서 위법성이 조각된다고 할 것이므로, 이 사건 비방금지 약정 위반에 따른 채무불이행이 성립하지 않는다고 봄이 타당하다.
7: ['채무불이행/NNG, IN_CUSTOM_DICT']: 설령 피고의 이 부분 제보 내용이 원고의 명예나 신용을 훼손할